**Li, X., Song, D., Zhang, P., et al. (2021). Emotion recognition based on multi-channel 1D CNN-LSTM model in multi-modal physiological signals. IEEE Access, 9, 69276-69286.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Input

# === LOAD DATA ===
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)
X_eeg = np.array([entry['eeg_data'] for entry in data])     # [samples, 5, 512]
y = np.array([entry['labels'] for entry in data])           # [samples, ...]

# === BALANCE CLASSES ===
def balance_classes(X, y, label_idx):
    labels = y[:, label_idx]
    classes, counts = np.unique(labels, return_counts=True)
    max_count = np.max(counts)
    Xb, yb = [], []
    for c in classes:
        idxs = np.where(labels == c)[0]
        Xc = X[idxs]
        yc = y[idxs]
        if len(Xc) < max_count:
            X_over, y_over = resample(Xc, yc, replace=True, n_samples=max_count-len(Xc), random_state=42)
            Xc = np.concatenate([Xc, X_over])
            yc = np.concatenate([yc, y_over])
        Xb.append(Xc)
        yb.append(yc)
    return np.concatenate(Xb), np.concatenate(yb)

# Balance (1) valence, then (2) arousal on result
X_eeg_bal, y_bal_val = balance_classes(X_eeg, y, label_idx=1)
X_eeg_bal, y_bal = balance_classes(X_eeg_bal, y_bal_val, label_idx=2)

# === PREPARE INPUT FOR 1D CNN ===
# ([samples, 5, 512]) --> ([samples, 512, 5])
X_eeg_1d = np.transpose(X_eeg_bal, (0, 2, 1))

# y_bal is now balanced on both valence and arousal
y_valence = y_bal[:, 1]
y_arousal = y_bal[:, 2]

num_valence = int(np.max(y_valence)) + 1
num_arousal = int(np.max(y_arousal)) + 1
y_val_cat = to_categorical(y_valence, num_classes=num_valence)
y_aro_cat = to_categorical(y_arousal, num_classes=num_arousal)

# === SPLIT DATA ===
X_tr, X_te, yv_tr, yv_te, ya_tr, ya_te = train_test_split(
    X_eeg_1d, y_val_cat, y_aro_cat, test_size=0.2, random_state=42
)

# === STATE-OF-THE-ART 1D CNN (Li et al., 2021) ===
model_1d_val = Sequential([
    Input(shape=(512, 5)),
    Conv1D(64, 7, activation='relu', padding='same'),
    MaxPooling1D(2),
    BatchNormalization(),
    Conv1D(128, 5, activation='relu', padding='same'),
    MaxPooling1D(2),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_valence, activation='softmax')
])
model_1d_val.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_1d_val.summary()

model_1d_aro = Sequential([
    Input(shape=(512, 5)),
    Conv1D(64, 7, activation='relu', padding='same'),
    MaxPooling1D(2),
    BatchNormalization(),
    Conv1D(128, 5, activation='relu', padding='same'),
    MaxPooling1D(2),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_arousal, activation='softmax')
])
model_1d_aro.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_1d_aro.summary()

# === TRAIN/TEST FOR BOTH ===
print('--- Valence (Balanced) ---')
history = model_1d_val.fit(
    X_tr, yv_tr,
    validation_data=(X_te, yv_te),
    epochs=50,
    batch_size=128
)
loss, acc = model_1d_val.evaluate(X_te, yv_te, verbose=0)
print(f"1D CNN (EEG) Valence Accuracy: {acc*100:.2f}%")

print('--- Arousal (Balanced) ---')
history_aro = model_1d_aro.fit(
    X_tr, ya_tr,
    validation_data=(X_te, ya_te),
    epochs=50,
    batch_size=128
)
loss_aro, acc_aro = model_1d_aro.evaluate(X_te, ya_te, verbose=0)
print(f"1D CNN (EEG) Arousal Accuracy: {acc_aro*100:.2f}%")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 512, 64)        │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 256, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,142,730 (8.17 MB)

 Trainable params: 2,142,346 (8.17 MB)

 Non-trainable params: 384 (1.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 512, 64)        │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 256, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,142,730 (8.17 MB)

 Trainable params: 2,142,346 (8.17 MB)

 Non-trainable params: 384 (1.50 KB)

--- Valence (Balanced) ---
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.1329 - loss: 6.3801 - val_accuracy: 0.2234 - val_loss: 3.1585
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1763 - loss: 4.4427 - val_accuracy: 0.1915 - val_loss: 2.6798
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2802 - loss: 2.0850 - val_accuracy: 0.2128 - val_loss: 2.3989
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3102 - loss: 1.9591 - val_accuracy: 0.1702 - val_loss: 2.3377
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2961 - loss: 1.9185 - val_accuracy: 0.1915 - val_loss: 2.2914
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3581 - loss: 1.7561 - val_accuracy: 0.1702 - val_loss: 2.2529
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.3666 - loss: 1.6983 - val_accuracy: 0.2234 - val_loss: 2.1913
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4309 - loss: 1.5692 - val_accuracy: 0